<a href="https://colab.research.google.com/github/jakubmis1998/NeuralNetworks/blob/main/zeroRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [324]:
# 0 recognition

import keras.preprocessing.image as kpi
from matplotlib import pyplot as plt
import numpy as np
from IPython import display
import sys

def draw_plot(E, T, w, THETA):
  x1, x2 = -4.0, 4.0
  plt.cla()
  plt.title("Recognition")
  plt.xlim(x1, x2)
  plt.ylim(-10, 10)
  plt.scatter(E[:, 0], E[:, 1], c=T)
  plt.plot([x1, x2], [-w[0]/w[1] * x1 + THETA/w[1], -w[0]/w[1] * x2 + THETA/w[1]]);
  display.display(plt.gcf())
  display.clear_output(wait=True)

def split_into_2D(image_name):
  # 2D array with single pictures
  array_2D = np.array([])

  # Load the main image
  main_img = kpi.load_img(image_name)

  # Convert to numpy array
  all_images = kpi.img_to_array(main_img)  # RGB 0-255 values
  all_images /= 255  # RGB 0-1 values

  # 70 x 40 x 3 - H x W x RGB
  array_shape = all_images.shape
  rows, columns = 10, 8
  # 7 x 5 x 3 - H x W x RGB
  single_img_shape = (array_shape[0] // 10, array_shape[1] // 8, array_shape[2])

  # Split entire array with all digits to single images
  for i in range(rows):
    row = np.array([])
    for j in range(columns):
      # Create single 7x5x3 picture and remove RGB canal
      single_image = all_images[
        i*single_img_shape[0]:(i+1)*single_img_shape[0],
        j*single_img_shape[1]:(j+1)*single_img_shape[1]
      ][:, :, 0]
      # Append single 7x5 picture
      row = np.append(row, single_image)
    # Append 8 pictures (row of 7x5 single pictures)
    array_2D = np.append(array_2D, row)
  
  # Make 2D array with 7x5 pictures
  array_2D = array_2D.reshape(10, 8, 7, 5)
  print("Wymiar pojedynczego obrazka:", array_2D[0][0].shape)
  print("Wymiar pojedynczego wiersza:", array_2D[0].shape)
  print("Wymiar całej tablicy:", array_2D.shape)
  print("Ilość liczb:", array_2D.size // (7*5))

  return array_2D

def show_image(img):
  """Show 2D image from NumPy array"""
  plt.imshow(img)

if __name__ == "__main__":
  # An 2D array of single pictures
  images = split_into_2D('mnist.png')

  # 8 samples 2D of digits == 0
  A = images[0]
  # 72 samples 2D of digits != 0
  B = images[1:10]

  # Samples for perceptron learning
  E = np.zeros((10, 8, 7, 5))  # 80 samples
  E[0] = A  # 8 good samples
  E[1:10] = B  # 72 bad samples

  # True results for perceptron learning
  T = np.zeros((10, 8))
  T[0] = 1
  T[1:10] = -1

  # Perceptron learning variables
  iteration_count = 5000
  learning_rate = 0.1
  k = 80
  N = 35
  THETA = np.random.default_rng().uniform(-1, 1, 1)[0]  # Random THETA [-1, 1)
  w = np.random.default_rng().uniform(-1, 1, N).reshape(7, 5)  # Random weights [-1, 1)
  global_life_time = 0
  local_life_time = 0
  best_weights = w
  classified = np.full((10, 8), False)  # Classified digits

  # Perceptron learning
  for index in range(iteration_count):
    # Coordinates of random digit
    i = np.random.randint(9)  # Row
    j = np.random.randint(7)  # Column

    # Output - result of the neural operation on E[i][j] - single sample - digit
    O = 1 if np.dot(w.flatten(), E[i][j].flatten()) - THETA >= 0 else -1

    # Calculate error
    ERR = T[i][j] - O

    # Unclassified sample
    if ERR != 0:
      local_life_time = 0  # Reset local life time
      classified = np.full((10, 8), False)  # Reset classified samples
      # Weights and THETA calculate
      w += learning_rate * ERR * E[i][j]
      THETA -= learning_rate * ERR
      # draw_plot(E, T, w, THETA)  # Update plot
    else:
      # Classified sample
      # Increase life time
      local_life_time += 1
      classified[i][j] = True
      # Store current winner
      if global_life_time < local_life_time:
        global_life_time = local_life_time
        best_weights = w

        
  # print(classified)
  # show_image(images[9][0])

Wymiar pojedynczego obrazka: (7, 5)
Wymiar pojedynczego wiersza: (8, 7, 5)
Wymiar całej tablicy: (10, 8, 7, 5)
Ilość liczb: 80
[[ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True False]
 [False False False False False False False False]]
